In [ ]:
import pandas as pd
from datetime import datetime
from google.colab import files

# Function to validate date format (YYYY-MM-DD)
def is_valid_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

# Function to load weather data from uploaded CSV
def load_weather_data():
    print("📂 Please upload your CSV file (with Date, Temperature, Condition columns)")
    uploaded = files.upload()

    if not uploaded:
        print("❌ No file uploaded.")
        return None

    file_name = list(uploaded.keys())[0]

    try:
        df = pd.read_csv(file_name)

        # Check required columns
        required_columns = {'Date', 'Temperature', 'Condition'}
        if not required_columns.issubset(df.columns):
            print("❌ CSV must contain 'Date', 'Temperature', and 'Condition' columns.")
            return None

        # Validate dates
        invalid_dates = df[~df['Date'].apply(is_valid_date)]
        if not invalid_dates.empty:
            print("❌ The CSV contains invalid date formats. Please fix them.")
            return None

        return df

    except Exception as e:
        print(f"❌ Error reading CSV: {e}")
        return None

# Function to summarize and export only summary to CSV
def summarize_and_export(df):
    if df.empty:
        print("📭 No data to summarize.")
        return

    # Sort by date and check trend
    df_sorted = df.sort_values(by="Date")
    temp_trend = df_sorted["Temperature"].diff().fillna(0)
    trend = temp_trend.mean()
    trend_str = "increasing" if trend > 0 else "decreasing" if trend < 0 else "stable"

    # Create summary
    summary = {
        "Average Temperature (°C)": [df['Temperature'].mean()],
        "Max Temperature (°C)": [df['Temperature'].max()],
        "Min Temperature (°C)": [df['Temperature'].min()],
        "Temperature Std Dev": [df['Temperature'].std()],
        "Most Common Condition": [df['Condition'].mode()[0]],
        "Temperature Trend": [trend_str.capitalize()]
    }

    # Add condition counts
    condition_counts = df['Condition'].value_counts().to_dict()
    for cond, count in condition_counts.items():
        summary[f"{cond} Days"] = [count]

    summary_df = pd.DataFrame(summary)

    print("\n📊 Weather Summary:")
    print(summary_df.T)

    # Save only summary
    summary_file = "weather_summary_only.csv"
    summary_df.to_csv(summary_file, index=False)
    print(f"\n✅ Summary exported as '{summary_file}'.")

    files.download(summary_file)

# Run the tool
df = load_weather_data()
if df is not None:
    summarize_and_export(df)